In [190]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io

In [187]:
data = scipy.io.loadmat('HW3Data.mat')

In [188]:
def logProd(x):
    '''X here is the numpy array'''
    return np.sum(x)

In [189]:
Xtrain = data['XTrain'].toarray()
Ytrain = np.array(data['yTrain'])

In [195]:
xTest = data['XTest'].toarray()
yTest = np.array(data['yTest'])

In [217]:
xTrainSmall = data['XTrainSmall'].toarray()
yTrainSmall = np.array(data['yTrainSmall'])

In [166]:
# okay now i calculate my prior on y which is just bernoulli distribution
def yprior(Ytrain):
    n = Ytrain.shape[0]
    numones = Ytrain[Ytrain == 1]
    prob_yone = np.sum(numones) / float(n)
    # i can do this using 1-prob_yone but just for sanity checking i am doing the calculation
    numtwos = Ytrain == 2
    prob_ytwo = np.sum(numtwos) / float(n)
    return np.array([prob_yone, prob_ytwo])

In [161]:
def NB_XgivenY(Xtrain, Ytrain):
    econodata = Ytrain == 1
    oniondata = Ytrain == 2
    economist_extractor = np.tile(econodata, (1, Xtrain.shape[1]))
    economist_data = Xtrain[economist_extractor].reshape(np.sum(econodata),Xtrain.shape[1])
    onion_extractor = np.tile(oniondata, (1, Xtrain.shape[1]))
    onion_data = Xtrain[onion_extractor].reshape(np.sum(oniondata),Xtrain.shape[1])
    #adding +1 for the prior B(2,1)
    onion_prob = (np.sum(onion_data, axis=0) + 1) / float(np.sum(oniondata) + 1)
    econ_prob = (np.sum(economist_data, axis=0) + 1) / float(np.sum(econodata) + 1)
    onion_prob = onion_prob.reshape(1, onion_prob.shape[0])
    econ_prob = econ_prob.reshape(1, econ_prob.shape[0])
    return np.concatenate((econ_prob, onion_prob), axis=0)

In [171]:
p = yprior(Ytrain)
print p[0]

0.741379310345


In [170]:
D = NB_XgivenY(Xtrain, Ytrain)
print D

[[ 0.59164733  0.95591647  0.28538283 ...,  0.00232019  0.00232019
   0.00232019]
 [ 0.67549669  1.          0.39735099 ...,  0.01324503  0.01324503
   0.01324503]]


In [179]:
def NB_Classify(D, p, X):
    '''Here is m * V matrix where each row corresponds to a feature vector'''
    yhat = list()
    m = X.shape[0]
    for i in range(m):
        # work on ith example
        econ_pred = X[i, :] * D[0, :] + ((1 - X[i, :]) * (1 - D[0, :]))
        onion_pred = X[i, :] * D[1, :] + ((1 - X[i, :]) * (1 - D[1, :]))
        # the above operations generate the vector containing the prob value i need to multiply
        # to avoid numerical errors we venture into log space log(p(y==1)) * sum(log(prob_values))
        fecon_pred = np.log(p[0]) + np.sum(np.log(econ_pred))
        fonion_pred = np.log(p[1]) + np.sum(np.log(onion_pred))
        
        # compare which is bigger and append the result in yhat
        if fecon_pred > fonion_pred:
            yhat.append(1)
        else:
            yhat.append(2)
    return np.array(yhat)

In [184]:
prediction = NB_Classify(D, p, Xtrain)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log


In [208]:
def ClassificationError(yhat, ytruth):
    error = yhat - ytruth.reshape(np.size(yhat))
    return (np.sum(np.abs(error))) / float(np.size(yhat))

In [209]:
yhatTrain = NB_Classify(D, p, Xtrain)
yhatTest = NB_Classify(D, p, xTest)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log


In [215]:
trainError = ClassificationError(yhatTrain, Ytrain)
testError = ClassificationError(yhatTest, yTest)
print "testError is",testError, 'train error is', trainError

testError is 0.0206896551724 train error is 0.0


In [222]:
D = NB_XgivenY(xTrainSmall, yTrainSmall)
p = yprior(yTrainSmall)
yhatTrain = NB_Classify(D, p, xTrainSmall)
yhatTest = NB_Classify(D, p, xTest)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log


In [223]:
smalltrainError = ClassificationError(yhatTrain, yTrainSmall)
testError = ClassificationError(yhatTest, yTest)
print "Test Error is", testError, 'Train Error is', smalltrainError

Test Error is 0.275862068966 Train Error is 0.0965517241379
